## 安装依赖包

In [ ]:
!pip install scipy tenacity tiktoken termcolor openai requests pandas pypinyin

In [1]:
import os
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import pandas as pd
import ast

GPT_MODEL = "gpt-3.5-turbo"

### 定义工具函数

首先，让我们定义一些用于调用聊天完成 API 的实用工具，并维护和跟踪对话状态。

In [ ]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):

    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
from urllib.parse import urlencode

baidu_key = os.getenv("BAIDU_KEY")

# 定义天气的请求
@retry(wait=wait_random_exponential(multiplier=2, max=10), stop=stop_after_attempt(3))
def weather_request(district_id):
    base_url = 'https://api.map.baidu.com/weather/v1/'
    params = {
        'district_id': district_id,
        'ak': baidu_key,
        'output': 'json',
        'data_type': 'now'
    }

    url = f"{base_url}?{urlencode(params)}"

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = ast.literal_eval(requests.get(url).text)
        if response["status"] != 0:
            raise Exception(response["message"])
        # 返回服务器的响应
        location = response["result"]["location"]
        weather = response["result"]["now"]
        c_name = location["country"]
        c_name += location["city"] if location["province"] == location["city"] else location["province"] + location["city"]
        return f"{c_name + location['name']} 的天气情况：\n" \
               f"天气：{weather['text']}\n" \
               f"温度(摄氏度)：{weather['temp']}\n" \
               f"湿度：{weather['rh']}\n" \
               f"风级：{weather['wind_dir'] + weather['wind_class']}\n"

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to fetch location weather")
        print(f"Exception: {e}")
        return e


In [4]:
print(weather_request(110100))

中国北京市北京 的天气情况：
天气：中雨
温度(摄氏度)：26
湿度：98
风级：东风2级



In [ ]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [ ]:
import traceback
# 提供将城市名字对应到英文名称
from pypinyin import lazy_pinyin
from functools import wraps
import re

def log_error(func):
    @wraps(func)
    def execute(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"错误信息：{e}, {traceback.format_exc()}")
    return execute


# unicode编码范围判断是否包含中文
def contains_chinese(input_str):
    pattern = re.compile(r'[\u4e00-\u9fff]')
    return bool(pattern.search(input_str))

@log_error
def translate_city_to_eg(city_name):
    # 判断是否为中文
    if not contains_chinese(city_name):
        return city_name
    # 包含中文进行拼音转化
    result = lazy_pinyin(city_name)
    city_name_eg = ''
    for part in result:
        city_name_eg += part.title()
    return city_name_eg


In [ ]:
print(translate_city_to_eg("上海市"))

In [ ]:
# 准备城市地址，用于查询天气
df = pd.read_csv("data/weather_district_id.csv")
df["province_eg"] = df["province"].apply(translate_city_to_eg)
df["city_eg"] = df["city"].apply(translate_city_to_eg)
df["district_eg"] = df["district"].apply(translate_city_to_eg)
df["combined"] = (
    "China; " + df.province_eg.str.strip() +
    "; " + df.city_eg.str.strip() +
    "; " + df.district_eg.str.strip()
)
df.drop("province_eg", axis=1, inplace=True)
df.drop("city_eg", axis=1, inplace=True)
df.drop("district_eg", axis=1, inplace=True)
print(df)

In [ ]:
from openai.embeddings_utils import get_embedding
# 将城市信息进行embedding，存入到df["embedding"]中
embedding_model = "text-embedding-ada-002"
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [ ]:
# df.to_csv("data/weather_district_id_embeddings.csv")

In [ ]:
from openai.embeddings_utils import cosine_similarity

# 将城市名称进行embedding，通过余弦相似度匹配城市唯一标识
def search_district_id(df, city_name):
    if not city_name:
        raise Exception("查询城市唯一标识参数异常")
    city_name = translate_city_to_eg(city_name)
    product_embedding = get_embedding(city_name,engine=embedding_model)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    return df.sort_values("similarity", ascending=False).head(1)['district_geocode'].values[0]


In [ ]:
print(search_district_id(df, "Shanghai, China"))

In [ ]:
# 只获取当前天气
# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    }
]


这段代码首先定义了一个`messages`列表用来存储聊天的消息，然后向列表中添加了系统和用户的消息。

然后，它使用了之前定义的`chat_completion_request`函数发送一个请求，传入的参数包括消息列表和函数列表。

在接收到响应后，它从JSON响应中解析出助手的消息，并将其添加到消息列表中。

最后，它打印出 GPT 模型回复的消息。

**（如果我们询问当前天气，GPT 模型会回复让你给出更准确的问题。）**

In [ ]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

pretty_print_conversation(messages)

**(我们需要提供更详细的信息，以便于 GPT 模型为我们生成适当的函数和对应参数。)**

In [ ]:
type(assistant_message)

## 使用 GPT 模型生成函数和对应参数

下面这段代码先向messages列表中添加了用户的位置信息。

然后再次使用了chat_completion_request函数发起请求，只是这次传入的消息列表已经包括了用户的新消息。

在获取到响应后，它同样从JSON响应中解析出助手的消息，并将其添加到消息列表中。

最后，打印出助手的新的回复消息。

In [ ]:
# 向messages列表添加一条用户角色的消息，用户告知他们在苏格兰的格拉斯哥
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "我不告诉你"  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的新的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message["role"] == "assistant" and assistant_message.get("function_call"):
    city_name = ast.literal_eval(assistant_message["function_call"]["arguments"])["location"]
    district_id = search_district_id(df, city_name)
    print(weather_request(district_id))


# 将助手的新的回复消息添加到messages列表中
# messages.append(assistant_message)

pretty_print_conversation(messages)